In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline
COLOR_TREAT = "#2ecc71"
COLOR_NO_TREAT = "#e74c3c"
%config InlineBackend.figure_format = 'retina'

In [ ]:
lalonde = pd.read_csv('lalonde.csv')
#lalonde.set_index('id', drop=True, inplace=True)
lalonde.head()

## 1. Naive analysis

TODO make a plot more comparable

In [ ]:
def plot_distrib(s1, s2, title, xLabel, yLabel, ax=None):
    bins = np.histogram(s1)[1]
    sns.distplot(s1, kde=False, color=COLOR_NO_TREAT, norm_hist=True, ax=ax, bins=bins)
    sns.distplot(s2, kde=False, color=COLOR_TREAT, norm_hist=True, ax=ax, bins=bins)
    if ax is None:
        plt.title(title)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        plt.legend(['No treatment', 'Treatment'])
    else:
        ax.set_title(title)
        ax.set_xlabel(xLabel)
        ax.set_ylabel(yLabel)
        ax.legend(['No treatment', 'Treatment'])

In [ ]:
plt.figure(figsize=(12,5))
plot_distrib(s1=lalonde.re78[lalonde['treat'] == 0], s2=lalonde.re78[lalonde['treat'] == 1], title='Distribution of the revenues', ax=None, xLabel='Revenue in 1978', yLabel='Number of persons (Density)')

Let's compare the average earnings of people that have/have not participated the job training:

In [ ]:
plt.figure(figsize=(20,5))
axes = plt.subplot(121)
lalonde.groupby(['treat'])['re78'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Mean salary with/without training')
plt.ylabel('Mean salaray')
plt.xlabel('Traning')
plt.subplot(122)
lalonde.groupby(['treat'])['re78'].median().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Median salary with/without training')
plt.ylabel('Median Salary')
plt.xlabel('Training')

## 2. A closer look at the data

In [ ]:
# distinguish categorical from non-categorical features
sns.set(font_scale=1.2)
lalonde_cat = lalonde[['black', 'hispan', 'married', 'nodegree', 'treat']]
lalonde_non_cat = lalonde[['age', 'educ', 're74', 're75', 're78', 'treat']]

### For each feature, compare its distribution in the treated group with its distribution in the control group

In [ ]:
f, axarr = plt.subplots(3, 3, figsize=(15, 18))
for index, feature in enumerate(['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']):
    ax = axarr[int(index/3)][index%3]
    plot_distrib(s1=lalonde[feature][lalonde['treat'] == 0], s2=lalonde[feature][lalonde['treat'] == 1], title=feature, xLabel = feature, yLabel='Density', ax=ax)

### Pairwise analysis of features' distribution
Comparing the distrbution of features for each distinct feature tuple can give additional information:
**TODO : décrire les obs**

In [ ]:
sns.pairplot(lalonde_non_cat, hue='treat', palette={0:"#e74c3c", 1: "#2ecc71"})

## 3. A propsensity score model
We should fit our model on the pre treatment features, though we will have to remove the re78 feature.

In [ ]:
import sklearn.linear_model

lal = lalonde.drop(['id','treat','re78'],1)
lal = preprocessing.scale(lal)
model = sklearn.linear_model.LogisticRegression()
model.fit(lal, lalonde.treat)
pred = model.predict_proba(lal)

In [ ]:
sum(model.predict(lal) == lalonde.treat)/len(lal)

In [ ]:
lalonde['pred'] = pred[:,1]
ax = sns.stripplot(x='id', y='pred', hue='treat', data=lalonde, palette={0:"#e74c3c", 1: "#2ecc71"})

## 4. Balancing the dataset via matching
Use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group. (Hint: you may explore the networkx package in Python for predefined matching functions.)

Your matching should maximize the similarity between matched subjects, as captured by their propensity scores. In other words, the sum (over all matched pairs) of absolute propensity-score differences between the two matched subjects should be minimized.

After matching, you have as many treated as you have control subjects. Compare the outcomes (re78) between the two groups (treated and control).

Also, compare again the feature-value distributions between the two groups, as you've done in part 2 above, but now only for the matched subjects. What do you observe? Are you closer to being able to draw valid conclusions now than you were before?

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite
#graph_size = lalonde['treat'].value_counts()
#G = nx.complete_bipartite_graph(graph_size[0], graph_size[1])
#G.add_nodes_from(lalonde['id'][lalonde.treat == 0], bipartite=0) # Add the node attribute "bipartite"
#G.add_nodes_from(lalonde['id'][lalonde.treat == 1], bipartite=1)

G=nx.Graph()
G.add_nodes_from(lalonde['id'][lalonde.treat == 0]) # Add the node attribute "bipartite"
G.add_nodes_from(lalonde['id'][lalonde.treat == 1])


In [ ]:
for ID_u, score_u in zip(lalonde.id[lalonde.treat == 0], lalonde.pred[lalonde.treat == 0]):
    for ID_v, score_v in zip(lalonde.id[lalonde.treat == 1], lalonde.pred[lalonde.treat == 1]):
        G.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

In [ ]:
#G.number_of_nodes()
G.number_of_edges()
#nx.draw(G)

In [ ]:
from networkx.algorithms import max_weight_matching
matching = max_weight_matching(G, maxcardinality=True)

In [ ]:
res = dict()
for key in matching.keys():
    if(key[0] == 'N'):
        res[key] = matching[key]
res

In [ ]:
lalonde_treated = lalonde[lalonde['treat'] == 1]
lalonde_treated['temp'] = 1
#lalonde_treated

lalonde_nt = lalonde[lalonde['treat'] == 0]
lalonde_nt['temp'] = 1
#lalonde_nt

result = pd.merge(lalonde_treated, lalonde_nt, on='temp')
result = result[['id_x' , 'id_y' , 'pred_x' , 'pred_y']]
result['diff'] = abs(result['pred_x'] - result['pred_y'])
result = result.set_index(['id_x', 'id_y'])
sum(result.loc[list(res.items())]['diff'])


In [ ]:
result.loc[list(res.items())]

In [ ]:
lalonde.set_index('id' , inplace = True)
matched = lalonde.loc[list(matching.keys())]


In [ ]:
matched = lalonde.loc[list(matching.keys())]
plt.figure(figsize=(20,5))
plt.subplot(121)
plt.title('Mean salary with/without training')
plt.ylabel('Mean salaray')
matched.groupby(['treat'])['re78'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.subplot(122)
plt.title('Median salary with/without training')
plt.ylabel('Median Salary')
matched.groupby(['treat'])['re78'].median().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])

In [ ]:
plot_distrib(s1=matched.re78[matched['treat'] == 0], s2=matched.re78[matched['treat'] == 1], 
             title='Distribution of the revenues', ax=None, xLabel='Revenue in 1978', yLabel='Number of persons (Density)')

In [ ]:
f, axarr = plt.subplots(3, 3, figsize=(15, 18))
for index, feature in enumerate(['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']):
    ax = axarr[int(index/3)][index%3]
    plot_distrib(s1=matched[feature][matched['treat'] == 0], s2=matched[feature][matched['treat'] == 1], title=feature, xLabel = feature, yLabel='Density', ax=ax)

In [ ]:
from scipy import stats
for feature in ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']:
    ks = stats.ks_2samp(matched[feature][matched['treat'] == 0], matched[feature][matched['treat'] == 1])
    print('For ' + str(feature)+ ' the KS stat and P-values are ' +str(ks))

## 5. Balancing the groups further
The "balanced" mode uses the values of y to automatically adjust
    weights inversely proportional to class frequencies in the input data
    as ``n_samples / (n_classes * np.bincount(y))`

In [ ]:
lalonde = pd.read_csv('lalonde.csv')
lal = lalonde.drop(['id','treat','re78'],1)
lal = preprocessing.scale(lal)
model = sklearn.linear_model.LogisticRegression()
model.fit(lal, lalonde.treat)
pred = model.predict_proba(lal)
lalonde['pred'] = pred[:,1]


Gprime=nx.Graph()
Gprime.add_nodes_from(lalonde['id'][lalonde.treat == 0])
Gprime.add_nodes_from(lalonde['id'][lalonde.treat == 1])

lalonde_not_black = lalonde[lalonde.black == 0]
for ID_u, score_u in zip(lalonde_not_black.id[lalonde_not_black.treat == 0], lalonde_not_black.pred[lalonde_not_black.treat == 0]):
    for ID_v, score_v in zip(lalonde_not_black.id[lalonde_not_black.treat == 1], lalonde_not_black.pred[lalonde_not_black.treat == 1]):
        Gprime.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

lalonde_black = lalonde[lalonde.black == 1]
for ID_u, score_u in zip(lalonde_black.id[lalonde_black.treat == 0], lalonde_black.pred[lalonde_black.treat == 0]):
    for ID_v, score_v in zip(lalonde_black.id[lalonde_black.treat == 1], lalonde_black.pred[lalonde_black.treat == 1]):
        Gprime.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

In [ ]:
matching = max_weight_matching(Gprime, maxcardinality=True)

In [ ]:
matched

In [ ]:
lalonde.set_index('id' , inplace = True)
matched = lalonde.loc[list(matching.keys())]

f, axarr = plt.subplots(3, 3, figsize=(15, 18))
for index, feature in enumerate(['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']):
    ax = axarr[int(index/3)][index%3]
    plot_distrib(s1=matched[feature][matched['treat'] == 0], s2=matched[feature][matched['treat'] == 1], title=feature, xLabel = feature, yLabel='Density', ax=ax)

In [ ]:
from scipy import stats
for feature in ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']:
    ks = stats.ks_2samp(matched[feature][matched['treat'] == 0], matched[feature][matched['treat'] == 1])
    print('For ' + str(feature)+ ' the KS stat and P-values are ' +str(ks))

## 6. A less naive analysis

In [ ]:
plt.figure(figsize=(20,5))
axes = plt.subplot(121)
matched.groupby(['treat'])['re78'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Mean salary with/without training')
plt.ylabel('Mean salaray')
plt.xlabel('Traning')
plt.subplot(122)
matched.groupby(['treat'])['re78'].median().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Median salary with/without training')
plt.ylabel('Median Salary')
plt.xlabel('Training')

# Question 2: Applied ML
We are going to build a classifier of news to directly assign them to 20 news categories. Note that the pipeline that you will build in this exercise could be of great help during your project if you plan to work with text!

- Load the 20newsgroup dataset. It is, again, a classic dataset that can directly be loaded using sklearn. TF-IDF, short for term frequency–inverse document frequency, is of great help when if comes to compute textual features. Indeed, it gives more importance to terms that are more specific to the considered articles (TF) but reduces the importance of terms that are very frequent in the entire corpus (IDF). Compute TF-IDF features for every article using TfidfVectorizer. Then, split your dataset into a training, a testing and a validation set (10% for validation and 10% for testing). Each observation should be paired with its corresponding label (the article category).

- Train a random forest on your training set. Try to fine-tune the parameters of your predictor on your validation set using a simple grid search on the number of estimator "n_estimators" and the max depth of the trees "max_depth". Then, display a confusion matrix of your classification pipeline. Lastly, once you assessed your model, inspect the feature_importances_ attribute of your random forest and discuss the obtained results.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(subset='all')
newsgroups.target_names

Here are all the target names inside the newsgroup dataset
## 2.1 Load the data, vectorize it and split it into training set and training set

In [ ]:
from  sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data) 
print(np.shape(X))

Our dataset has 18846 datapoints and 173762 features. Now let split it into train, valid and test set. To do so, we created a function *get_train_valid_test_set* written in the box below.

In [ ]:
def get_train_valid_test_set(data, labels, perc_for_valid, perc_for_test):
    # returns the train, valid and test sets
    # data : n x nb_features matrix
    # perc_for_valid : value between 0 and 1 as the percentage of the dataset to be used in validation
    # perc_for_test : value between 0 and 1 as the percentage of the dataset to be used in testing
    
    uindx = np.random.permutation(np.shape(data)[0])
    X_shuffled = data[uindx]
    labels_shf = labels[uindx]
    
    # take perc_for_test% of the dataset for testing
    X_test_shf = X_shuffled[:int(X_shuffled.shape[0]*perc_for_test)]
    y_test_shf = labels_shf[:int(X_shuffled.shape[0]*perc_for_test)]
    
    # take perc_for_valid% of the dataset for validation
    X_valid_shf = X_shuffled[int(X_shuffled.shape[0]*perc_for_test):
                             int(X_shuffled.shape[0]*(perc_for_test + perc_for_valid))]
    y_valid_shf = labels_shf[int(X_shuffled.shape[0]*perc_for_test):
                             int(X_shuffled.shape[0]*(perc_for_test + perc_for_valid))]
    
    X_train_shf = X_shuffled[int(X_shuffled.shape[0]*(perc_for_test + perc_for_valid)):]
    y_train_shf = labels_shf[int(X_shuffled.shape[0]*(perc_for_test + perc_for_valid)):]
    
    return X_test_shf, X_train_shf, X_valid_shf, y_test_shf, y_train_shf, y_valid_shf

In [ ]:
X_test, X_train, X_valid, y_test, y_train, y_valid = get_train_valid_test_set(X, newsgroups.target, 0.1, 0.1)
print('test set size : '+ str(X_test.shape))
print('valid set size : '+ str(X_valid.shape))
print('train set size : '+ str(X_train.shape))

## 2.2 Classification
**Train a random forest** on your training set. Try to **fine-tune the parameters** of your predictor on your validation set using a simple grid search on the number of estimator "n_estimators" and the max depth of the trees "max_depth". Then, **display a confusion matrix** of your classification pipeline. Lastly, once you assessed your model, **inspect the feature_importances_** attribute of your random forest and discuss the obtained results.

### Fine Tune max depth and number of trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

number_trees = [125, 300, 500, 600 ]
max_depth = [5, 8, 12, 16, 20]

best_score = 0

#Loop for hyperparameter number_trees and max_depth
for nb_t in number_trees:
    for nb_d in max_depth:

        # Random forest model
        rand_forest_model = RandomForestClassifier(n_estimators=nb_t, max_depth=nb_d)
        rand_forest_model.fit(X_train, y_train)
        y_pred_val = rand_forest_model.predict(X_valid)

        # get score
        score = accuracy_score(y_valid, y_pred_val)
        print('loading : '+ str(stepinfo/(len(number_trees)*len(max_depth))) + '% nb_trees : ' + str(nb_t) + ' depth : '
              + str(nb_d) + ' score : ' + str(score))
        
        # update best score if needed
        if score > best_score:
            best_nb_trees = nb_t
            best_max_depth = nb_d
            best_score = score
print(best_score)       

### Display confusion matrix

In [ ]:
rand_forest_model = RandomForestClassifier(n_estimators=best_nb_trees, max_depth=best_max_depth)
rand_forest_model.fit(X_train, y_train)
y_pred_test = rand_forest_model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_pred_test, y_test)
#np.fill_diagonal(cm, 0)
plt.matshow(cm)
plt.title('Confusion matrix of the classifier')
plt.colorbar()
plt.show()


## TODO : regarder comment mettre la confusion matrix en % 
Faire la features importance
Analayser tout ca 